In [2]:
# from robot_controller import FrankaOSCController
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
gripper_pose = [
    [ 0.99987168, -0.00938568, -0.01221834,  0.69546502],
    [-0.00908794, -0.99965622,  0.02419956, -0.02291797],
    [-0.01244126, -0.02408542, -0.99963248,  0.00931603],
    [ 0.,          0.,          0.,          1.        ]]
gripper_pose = np.array(gripper_pose)

In [25]:
def estimate_tag_pose(finger_pose):
    """
    Estimate the tag pose given the gripper pose by applying the gripper-to-tag transformation.

    Args:
        finger_pose (eef_pose): 4x4 transformation matrix from gripper to robot base
    Returns:
        hand_pose: 4x4 transformation matrix from hand to robot base
        tag_pose: 4x4 transformation matrix from tag to robot base
    """
    from scipy.spatial.transform import Rotation

    # Estimate the hand pose
    # finger_to_hand obtained from the product manual: 
    # [https://download.franka.de/documents/220010_Product%20Manual_Franka%20Hand_1.2_EN.pdf]
    finger_to_hand = np.array([
        [0.707,  0.707, 0, 0],
        [-0.707, 0.707, 0, 0],
        [0, 0, 1, 0.1034],
        [0, 0, 0, 1],
    ])
    finger_to_hand = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0.1034],
        [0, 0, 0, 1],
    ])
    hand_to_finger = np.linalg.inv(finger_to_hand)
    print("hand to finger", hand_to_finger)
    hand_pose = np.dot(finger_pose, hand_to_finger)

    t_tag_to_hand = np.array([0.048914, 0.0275, 0.00753])
    # R_tag_to_hand = Rotation.from_quat([0.5, -0.5, 0.5, -0.5])
    R_tag_to_hand = Rotation.from_quat([0, 0, 0, 1])
    tag_to_hand = np.eye(4)
    tag_to_hand[:3, :3] = R_tag_to_hand.as_matrix()
    tag_to_hand[:3, 3] = t_tag_to_hand

    tag_pose = np.dot(hand_pose, tag_to_hand)
    
    return hand_pose, tag_pose

    t_gripper_to_tag_top_left = np.array([0.048914, 0.0275, 0.00753])
    R_gripper_to_tag_top_left = Rotation.from_quat([0.5, -0.5, 0.5, -0.5])

    gripper_pos, gripper_mat = gripper_pose[:3, 3], gripper_pose[:3, :3]
    gripper_rot = Rotation.from_matrix(gripper_mat)
    gripper_to_world_rot = gripper_rot.inv()

    print("gripper rot", gripper_rot.as_euler('xyz', degrees=True))
    transformed_delta_pos = t_gripper_to_tag_top_left.copy()
    transformed_delta_pos = R_gripper_to_tag_top_left.inv().apply(transformed_delta_pos)
    print("first transform", transformed_delta_pos)
    transformed_delta_pos = gripper_to_world_rot.apply(transformed_delta_pos)
    print("second transform", transformed_delta_pos)
    
    tag_pos = gripper_pos + transformed_delta_pos
    tag_pose = gripper_pose.copy()
    tag_pose[:3, 3] = tag_pos
    # R_gripper_to_tag_top_left = Rotation.from_quat([0, 0, 0, 1]).as_matrix()
    # T_gripper_to_tag_top_left = np.eye(4)
    # T_gripper_to_tag_top_left[:3, :3] = R_gripper_to_tag_top_left
    # T_gripper_to_tag_top_left[:3, 3] = t_gripper_to_tag_top_left

    # tag_pose = gripper_pose @ T_gripper_to_tag_top_left
    return tag_pose

hand_pose, tag_pose = estimate_tag_pose(gripper_pose)
print(f"Tag pos: {tag_pose[:3, 3]}")
print(f"Finger pos: {gripper_pose[:3, 3]}")
print(f"Hand pos: {hand_pose[:3, 3]}")

# Visualize the tag and gripper poses
vis = o3d.visualization.Visualizer()
robot_base_coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4)
hand_coord = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.2, origin=hand_pose[:3, 3])
hand_coord.rotate(hand_pose[:3, :3])
gripper_coord = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.1, origin=gripper_pose[:3, 3])
gripper_coord.rotate(gripper_pose[:3, :3])
tag_coord = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.05, origin=tag_pose[:3, 3])
tag_coord.rotate(tag_pose[:3, :3])

o3d.visualization.draw_geometries([robot_base_coord, hand_coord, gripper_coord, tag_coord])

hand to finger [[ 0.70721358 -0.70721358  0.          0.        ]
 [ 0.70721358  0.70721358  0.          0.        ]
 [ 0.          0.          1.         -0.1034    ]
 [ 0.          0.          0.          1.        ]]
Tag pos: [ 0.71127151 -0.07939805  0.10366078]
Finger pos: [ 0.69546502 -0.02291797  0.00931603]
Hand pos: [ 0.6967284  -0.0254202   0.11267803]


In [ ]:
Finger pos: [ 0.69546502 -0.02291797  0.00931603]
Hand pos: [ 0.69546502 -0.02291797  0.11271603]